In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import optuna

In [14]:
if torch.cuda.is_available():
    print("GPU is available!")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available. Using CPU.")

GPU is available!
Using GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.cuda.is_available()
print(device)


True


In [6]:
DATA_PATH = r"C:\Users\Lenovo\OneDrive\Desktop\deep Learning\pyTorch\data"

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST(
    root=DATA_PATH,
    train=True,
    download=False,
    transform=transform
)

test_dataset = datasets.FashionMNIST(
    root=DATA_PATH,
    train=False,
    download=False,
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
class CNN(nn.Module):
    def __init__(self, activation, use_bn, dropout):
        super().__init__()

        act = {
            "relu": nn.ReLU(),
            "sigmoid": nn.Sigmoid(),
            "tanh": nn.Tanh()
        }[activation]

        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3),
            act,
            nn.Conv2d(32, 64, 3),
            act,
            nn.MaxPool2d(2),
            nn.BatchNorm2d(64) ,
            nn.Dropout(dropout)
        )

        self.fc = nn.Sequential(
            nn.Linear(64 * 12 * 12, 128),
            act,
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


In [8]:
class MLP(nn.Module):
    def __init__(self, layers, activation, use_bn, dropout):
        super().__init__()

        act = {
            "relu": nn.ReLU(),
            "sigmoid": nn.Sigmoid(),
            "tanh": nn.Tanh()
        }[activation]

        modules = []
        in_features = 28 * 28

        for units in layers:
            modules.append(nn.Linear(in_features, units))
            if use_bn:
                modules.append(nn.BatchNorm1d(units))
            modules.append(act)
            modules.append(nn.Dropout(dropout))
            in_features = units

        modules.append(nn.Linear(in_features, 10))
        self.net = nn.Sequential(*modules)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.net(x)


In [9]:
def train(model, optimizer, criterion, epochs):
    model.train()
    history = {"loss": [], "acc": []}

    for _ in range(epochs):
        correct, total, loss_sum = 0, 0, 0

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, y)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()
            _, pred = out.max(1)
            correct += pred.eq(y).sum().item()
            total += y.size(0)

        history["loss"].append(loss_sum / len(train_loader))
        history["acc"].append(correct / total)

    return history


def test(model):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            _, pred = out.max(1)
            correct += pred.eq(y).sum().item()
            total += y.size(0)

    return correct / total


In [ ]:
def objective(trial):

    model_type = trial.suggest_categorical("model", ["cnn", "mlp"])
    activation = trial.suggest_categorical("activation", ["relu", "sigmoid", "tanh"])
    optimizer_name = trial.suggest_categorical("optimizer", ["SGD", "Adam"])
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    use_bn = trial.suggest_categorical("batchnorm", [True, False])
    epochs = trial.suggest_int("epochs", 5, 15)

    if model_type == "cnn": 
        model = CNN(activation, use_bn, dropout).to(device)
    else:
        layers = trial.suggest_categorical("layers", [[256], [512,256,128]])
        model = MLP(layers, activation, use_bn, dropout).to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr) if optimizer_name == "Adam" \
        else optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    criterion = nn.CrossEntropyLoss()

    train(model, optimizer, criterion, epochs)
    acc = test(model)

    return acc


In [12]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

print("Best Accuracy:", study.best_value)
print("Best Params:", study.best_params)


[I 2026-01-30 14:18:53,046] A new study created in memory with name: no-name-375672a2-6094-4540-9184-80716a6a5e05
[I 2026-01-30 14:21:36,411] Trial 0 finished with value: 0.8996 and parameters: {'model': 'cnn', 'activation': 'tanh', 'optimizer': 'SGD', 'lr': 0.0003178330254512856, 'dropout': 0.24404087376846434, 'batchnorm': True, 'epochs': 10}. Best is trial 0 with value: 0.8996.
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [256] which is of type list.
  optuna_warn(message)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [512, 256, 128] which is of type list.
  optuna_warn(message)
[I 2026-01-30 14

Best Accuracy: 0.8996
Best Params: {'model': 'cnn', 'activation': 'tanh', 'optimizer': 'SGD', 'lr': 0.0003178330254512856, 'dropout': 0.24404087376846434, 'batchnorm': True, 'epochs': 10}
